<a href="https://colab.research.google.com/github/BhanuDanda/AIML-/blob/main/assignment%203-part%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
print(3.1)
class Board:
    def __init__(self, height=3, width=3, to_move='X', utility=0):
        self.height = height
        self.width = width
        self.to_move = to_move
        self.utility = utility
        self.squares = {}

    def new(self, moves, to_move):
        new_board = Board(self.height, self.width, to_move)
        new_board.squares = self.squares.copy()
        new_board.squares.update(moves)
        return new_board

    def __str__(self):
        board_str = ''
        for y in range(self.height):
            for x in range(self.width):
                board_str += self.squares.get((x, y), '.') + ' '
            board_str += '\n'
        return board_str


class Game:
    pass


class TicTacToe(Game):

    def __init__(self, height=3, width=3, k=3):
        self.k = k  # k in a row
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='X', utility=0)

    def actions(self, board):
        return self.squares - set(board.squares)

    def result(self, board, square):
        player = board.to_move
        board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        win = self.k_in_row(board, player, square, self.k)
        board.utility = (0 if not win else +1 if player == 'X' else -1)
        return board

    def utility(self, board, player):
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        return board.utility != 0 or len(self.actions(board)) == 0

    def display(self, board):
        print(board)

    def k_in_row(self, board, player, square, k):
        def in_row(x, y, dx, dy):
            if not (0 <= x < board.width and 0 <= y < board.height):
                return 0
            if board.squares.get((x, y)) != player:
                return 0
            return 1 + in_row(x + dx, y + dy, dx, dy)

        return any(
            in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
            for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1))
        )


def play_game():
    game = TicTacToe(height=3, width=3, k=3)
    board = game.initial
    while not game.is_terminal(board):
        game.display(board)
        available_actions = game.actions(board)
        print(f"Available actions: {available_actions}")

        move = input(f"{board.to_move}'s turn. Enter your move as 'x,y': ")
        try:
            x, y = map(int, move.split(','))
            if (x, y) in available_actions:
                board = game.result(board, (x, y))
            else:
                print("Invalid move. Try again.")
        except ValueError:
            print("Invalid input. Please enter your move in 'x,y' format.")

    game.display(board)
    if board.utility == 1:
        print("X wins!")
    elif board.utility == -1:
        print("O wins!")
    else:
        print("It's a draw!")


if __name__ == "__main__":
    play_game()


3.1
. . . 
. . . 
. . . 

Available actions: {(0, 1), (1, 2), (2, 1), (0, 0), (1, 1), (2, 0), (0, 2), (2, 2), (1, 0)}
X's turn. Enter your move as 'x,y': 1,1
. . . 
. X . 
. . . 

Available actions: {(0, 1), (1, 2), (2, 1), (0, 0), (2, 0), (0, 2), (2, 2), (1, 0)}
O's turn. Enter your move as 'x,y': 0,1
. . . 
O X . 
. . . 

Available actions: {(1, 2), (2, 1), (0, 0), (2, 0), (0, 2), (2, 2), (1, 0)}
X's turn. Enter your move as 'x,y': 1,2
. . . 
O X . 
. X . 

Available actions: {(2, 1), (0, 0), (2, 0), (0, 2), (2, 2), (1, 0)}
O's turn. Enter your move as 'x,y': 2,1
. . . 
O X O 
. X . 

Available actions: {(0, 0), (2, 0), (0, 2), (2, 2), (1, 0)}
X's turn. Enter your move as 'x,y': 0,0
X . . 
O X O 
. X . 

Available actions: {(1, 0), (0, 2), (2, 0), (2, 2)}
O's turn. Enter your move as 'x,y': 1,0
X O . 
O X O 
. X . 

Available actions: {(0, 2), (2, 0), (2, 2)}
X's turn. Enter your move as 'x,y': 0,2
X O . 
O X O 
X X . 

Available actions: {(2, 0), (2, 2)}
O's turn. Enter your move as

In [7]:
from collections import defaultdict
print(3.2)
class Board(defaultdict):

    empty = '.'
    off = '#'

    def __init__(self, width=8, height=8, to_move=None, **kwds):
        super().__init__(lambda: self.empty)
        self.width = width
        self.height = height
        self.to_move = to_move
        self.update(**kwds)

    def new(self, changes: dict, **kwds) -> 'Board':
        board = Board(width=self.width, height=self.height, to_move=self.to_move, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def missing(self, loc):
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self[x, y]
        else:
            return self.off

    def __hash__(self):
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

    def __repr__(self):
        def row(y):
            return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) + '\n'

if __name__ == "__main__":
    board = Board()
    print("Initial Board:")
    print(board)

    board[0, 0] = 'X'
    board[1, 1] = 'O'
    print("Board after moves:")
    print(board)

    new_board = board.new({(1, 1): 'X'})
    print("New Board with changes:")
    print(new_board)

    print("Missing at (2, 2):", new_board.missing((2, 2)))  # should return '.'
    print("Missing at (8, 8):", new_board.missing((8, 8)))  # should return '#'

    print("Hash of the new board:", hash(new_board))


3.2
Initial Board:
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .

Board after moves:
X . . . . . . .
. O . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .

New Board with changes:
X . . . . . . .
. X . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .

Missing at (2, 2): .
Missing at (8, 8): #
Hash of the new board: -5786254831368673547


In [9]:
import random
print(3.3)
class Game:
    def __init__(self):
        self.state = 0

    def actions(self, state):
        return ["move left", "move right"]

    def result(self, state, action):
        if action == "move left":
            return state - 1
        elif action == "move right":
            return state + 1
        return state

    def is_terminal(self, state):
        return state <= -5 or state >= 5  # Example terminal conditions

def random_player(game, state):
    return random.choice(list(game.actions(state)))

def simple_search_algorithm(game, state):
    actions = game.actions(state)
    best_action = actions[0]
    best_result = game.result(state, best_action)

    for action in actions:
        result = game.result(state, action)
        if result > best_result:
            best_result = result
            best_action = action

    return best_action

def player(search_algorithm):
    return lambda game, state: search_algorithm(game, state)

if __name__ == "__main__":
    game_instance = Game()
    current_state = game_instance.state

    action_random = random_player(game_instance, current_state)
    print(f"Random Player Action: {action_random}")

    action_search = player(simple_search_algorithm)(game_instance, current_state)
    print(f"Search Algorithm Player Action: {action_search}")

    current_state = game_instance.result(current_state, action_random)
    print(f"New State after Random Action: {current_state}")

    current_state = game_instance.result(current_state, action_search)
    print(f"New State after Search Action: {current_state}")

    if game_instance.is_terminal(current_state):
        print("Game Over!")
    else:
        print("Game continues...")


3.3
Random Player Action: move right
Search Algorithm Player Action: move right
New State after Random Action: 1
New State after Search Action: 2
Game continues...
